In [20]:
import os
# Initialize embedding model using OpenAI
from langchain.embeddings import OpenAIEmbeddings
# Set up Astra DB vector store with embeddings
from langchain_astradb import AstraDBVectorStore
# Initialize the OpenAI chat model
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [21]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
# Initialize embedding model using OpenAI
from langchain.embeddings import OpenAIEmbeddings
# Set up Astra DB vector store with embeddings
from langchain_astradb import AstraDBVectorStore
# Initialize the OpenAI chat model
from langchain.chat_models import ChatOpenAI


# Load your OpenAI API key securely (e.g., from environment variable)
OPENAI_API_KEY = "<OPENAI_API_KEY>"  # replace with os.getenv("OPENAI_API_KEY") in practice
# Define your Astra DB API endpoint
ASTRA_DB_API_ENDPOINT = "<ASTRA_DB_API_ENDPOINT>"
# Define your Astra DB application token
ASTRA_DB_APPLICATION_TOKEN = "<ASTRA_DB_APPLICATION_TOKEN>"
# Specify the collection name to use in Astra DB
COLLECTION_NAME = "mdvs"


# Load your OpenAI API key securely (e.g., from environment variable)
OPENAI_API_KEY = "<OPENAI_API_KEY>"  # replace with os.getenv("OPENAI_API_KEY") in practice
# Set up Astra DB vector store with embeddings
vector_store = AstraDBVectorStore(
    embedding=embeddings,
# Specify the collection name to use in Astra DB
    collection_name=COLLECTION_NAME,
# Define your Astra DB application token
ASTRA_DB_APPLICATION_TOKEN = "<ASTRA_DB_APPLICATION_TOKEN>"
# Define your Astra DB API endpoint
ASTRA_DB_API_ENDPOINT = "<ASTRA_DB_API_ENDPOINT>"
)


query = "How do I use a CPAP machine?"
results = vector_store.similarity_search_with_score(query, k=3)
print("\n--- Similarity Search with Score ---")
for doc, score in results:
    print(f"[SIM={score:.3f}] {doc.page_content[:300]}... [metadata={doc.metadata}]")


custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful assistant. 
If the exact instructions are not explicitly written, try to infer based on related setup or usage instructions.
Never say "I don't know" — always give your best possible answer.

Context:
{context}

Question:
{question}

Answer:
"""
)


retriever = vector_store.as_retriever(search_kwargs={"k": 20})
# Initialize the OpenAI chat model
llm = ChatOpenAI(
# Load your OpenAI API key securely (e.g., from environment variable)
OPENAI_API_KEY = "<OPENAI_API_KEY>"  # replace with os.getenv("OPENAI_API_KEY") in practice
    model_name="gpt-4",
    temperature=0
)

# Build the retrieval-based QA pipeline using LLM and retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": custom_prompt}
)


result = qa_chain({"query": query})

print("\n--- RetrievalQA Output ---")
print(f"Question: {query}")
print("Answer:", result["result"])

print("\n--- Source Documents Used ---")
for doc in result["source_documents"]:
    print(doc.page_content[:500])  
    print("---")



--- Similarity Search with Score ---
[SIM=0.519] [Hemodialysis at Home](https://www.medviewsystems.com/hemodialysis/) *
 [Medication](https://www.medviewsystems.com/medications-management/)     *   [IV
 Pumps](https://www.medviewsystems.com/instructions-for-iv-pumps/)     *   [Medication
 Preparations](https://www.medviewsystems.com/instructions-f... [metadata={'source': 'C:\\Users\\hahaha\\Downloads\\mdvc.txt', 'filename': 'C:\\Users\\hahaha\\Downloads\\mdvc.txt'}]
[SIM=0.518] Preparations](https://www.medviewsystems.com/instructions-for-medication-preparations/)
 *   [Catheters](https://www.medviewsystems.com/instructions-for-cathete
 Chunk 499 | Source: https://freestyleserver.com/Payloads/IFU/2017july/ART22813-003_rev
A_Web.pdf  rs/)     *   [Nutritional Support
 (TPN)... [metadata={'source': 'C:\\Users\\hahaha\\Downloads\\mdvc.txt', 'filename': 'C:\\Users\\hahaha\\Downloads\\mdvc.txt'}]
[SIM=0.517] A_Web.pdf  //www.medviewsystems.com/instructions-for-medication-preparations/)    